# Company Summarizer

LLMs are great at structuring and summarizing unstructured data, such as natural language. We can leverage LLMs to extract specific information from text content, such as websites.

Imagine you are an angel investor or you are hunting for a job, but you are searching for a specific traits in companies, such as an innovative culture that embraces experimentation. Or you want to get an overview about their team, founders or investors. LLMs are perfect for such tasks. They make automization of extracting information from natural language super easy. And they are really good at it, too.

In this project, I am using the OpenAI API to ask LLMs to extract and structure specific data from company websites. I will try to find out about their product, their culture and their founder(s).


In [169]:
from typing import List, Generator
import json
import os

from openai import OpenAI
from dotenv import load_dotenv
from bs4 import BeautifulSoup 
from IPython.display import Markdown, display
import requests
import gradio as gr

First, lets create a Website class that uses requests and BeautifulSoup to scrape contents from a company website

In [170]:
class Website:
    url: str
    title: str
    body: str
    text: str
    links: List[str]
    
    def __init__(self, url: str):
        self.url = url
        self.__scrape_webpage()

    def __scrape_webpage(self):
        response = requests.get(self.url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.__get_website_title(soup)
        self.__get_website_text(soup)
        self.__get_links_from_website(soup)

    def __get_website_title(self, soup: BeautifulSoup):
        self.title = soup.title.string if soup.title else "Website has no title"
        
    def __get_website_text(self, soup: BeautifulSoup):
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
            
    def __get_links_from_website(self, soup: BeautifulSoup):
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Website Title:\n{self.title}\n\nWebsite Contents:\n{self.text}\n\n"
        

Let's give this a try for the company website of my friends at Plancraft - a company creating a software for craftsmen to organize their projects, plan their capacities, create offers and bills etc.

In [171]:
plancraft = Website("https://plancraft.de/en")

In [172]:
print(plancraft.get_contents())

Website Title:
Boost your contractor business with plancraft - Easy Craftsman Software

Website Contents:
Features
Order Documents
Offers, order confirmations, delivery notes and invoices.
Craftsman Cloud
Access your data from anywhere and on any device thanks to the cloud.
Interfaces
Import your data easily.
Mobile Measurement
Take measurements on site at the customer's premises? No problem!
Financial Administration
Manage payments from wherever you want.
Time Recording
All working hours at a glance.
Master Data
Manage your data easily.
Post-Calculation
Determine profitable prices for your services.
Planning Board
Project and deployment planning for your company.
Pricing
Crafts
Building Contractor
Learn how Plancraft can help you optimize your construction business.
Roofer
Find out how digitization can help you as a roofer
‍
Painter
An overview for Plancraft in painting and varnishing companies
‍
Joiner
Discover Plancraft's options for your joinery
Carpenter
Learn how to improve your 

## Leveraging GPT-4o-mini to filter relevant links 

To gather information about what the company does, what the culture looks like and whether they have any open tech positions, I will call the OpenAI API to return a JSON-formatted overview of relevant links.

Let's load the OpenAI API key from my local .env file and create the OpenAI client first.

In [173]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

openai = OpenAI()

 Now, let's create an OpenAI caller function to handle single-shot calls to the OpenAI API that returns Python dicts by parsing the JSON response of the GPT model. Note: We will have to tell the model both in the system and user prompt to return JSON-formatted data. 

In [174]:
def call_openai_json(system_prompt: str, user_prompt: str) -> dict:
    response=openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format={"type": "json_object"}
    )
    json_str = response.choices[0].message.content
    return json.loads(json_str)

## Creating promts to get relevant links for the question at hand in JSON format

In [175]:
system_prompt_links = """
You are a web scraper assistant and are provided with a list of links from a company website. You are able to select relevant links 
from that list to retrieve information about the company's product, its culture and its founders. 

You will respond using JSON, using the following format:

{
    "relevant_links": [
        "type": "about", "url": "https://some-company.com/about-us",
        "type": "product", "url": "https://cool-startup.io/features",
        "type": "jobs", "url": "https://cool-startup.io/jobs",
        ...
    ]
}
"""

def create_user_prompt_links(website: Website) -> str:
    return f"""
    Below, you will find a list of links from the company website {website.url}.
    Please decide which links are useful to find information about the company's product, its culture and its founders. 
    Answer in JSON format and return full HTTPS URLs. Don't return links for stuff like terms of service, legal or contact. 

    These are the links: {website.links}
    """

In [176]:
print(create_user_prompt_links(plancraft))


    Below, you will find a list of links from the company website https://plancraft.de/en.
    Please decide which links are useful to find information about the company's product, its culture and its founders. 
    Answer in JSON format and return full HTTPS URLs. Don't return links for stuff like terms of service, legal or contact. 

    These are the links: ['/en', '/en/features', '/en/features', '/en/features', '/en/features', '/en/features', '/en/features', '/en/features', '/en/features/master-data', '/en/features', '/en/features/project-planning', '/en/prices', '/en/crafts', '/en/crafts/construction-companies', '/en/crafts/roofer', '/en/crafts/painter-and-varnisher', '/en/crafts/joinery', '/en/crafts/carpentry', '/en/success-stories', '/en/success-stories/painter-andy', '/en/success-stories/carpentry-krumbuegel', '/en/success-stories/dennis-hoeper', '/en/success-stories/sebastian-bluemel', '/en/success-stories/giuseppe-giannino', '/en/success-stories/martin-bruno-bienek', '/en/s

## Calling GPT-4o-mini to do the repetetive work for us

In [180]:
user_prompt_links = create_user_prompt_links(website = plancraft)
links = call_openai_json(
    system_prompt=system_prompt_links,
    user_prompt=user_prompt_links
)
print(json.dumps(links, indent=4))

{
    "relevant_links": [
        {
            "type": "about",
            "url": "https://plancraft.de/en/team"
        },
        {
            "type": "product",
            "url": "https://plancraft.de/en/features"
        },
        {
            "type": "product",
            "url": "https://plancraft.de/en/demo"
        },
        {
            "type": "product",
            "url": "https://plancraft.de/en/prices"
        },
        {
            "type": "culture",
            "url": "https://plancraft.de/en/career"
        },
        {
            "type": "success_stories",
            "url": "https://plancraft.de/en/success-stories"
        },
        {
            "type": "blog",
            "url": "https://plancraft.de/en/blog"
        }
    ]
}


## Getting the information from these links using our Website class
Awesome! Now, let's scrape the information from these links so we can create our company overview.

In [161]:
def scrape_links(homepage: Website, links: List[str]) -> List[Website]:
    websites = [homepage]
    for website in links["relevant_links"]:
        websites.append(Website(website["url"]))
    return websites

websites = scrape_links(homepage = plancraft, links = links)
website_contents = "\n\n".join(website.get_contents() for website in websites)

## Letting GPT-4o-mini summarize and structure the information from unstructured data
As you can see, the contents of the different websites are quite unstructured. LLMs are really efficient at making sense of such data though. Let's create another system and user prompt to create a neatly formatted summary of the scraped information. We will ask the LLM to respond using markdown for nice formatting.

In [162]:
system_prompt_summary = """
You are a useful assistant that summarizes text from websites. You will be provided a list of website contents that was scraped
from multiple pages of a company website. For each site, you will get the website title and the text content. Your task is to extract information 
about the company's product, its culture and its founders. 

Answer using markdown to provide nice format and structure.
"""

def get_user_prompt_summary(url: str, website_contents: str) -> str: 
    return f"""
Below, you will find the contents of multiple pages scraped from this company webpage: {url}.
Use the contents to create a brief summary of the company's product, its culture and its founders. 
If available, include the names and contacts of the founders so I can get in touch with them.

Answer using markdown for nice formatting. 

These are the contents of multiple relevant webpages: {website_contents}
"""[:40000] # Truncate the response to not create too many tokens for the model at hand

Finally, let's call the LLM again with the new prompts. I will create a new function to call OpenAI that receives plain text rather than JSON.

In [163]:
def call_openai(system_prompt: str, user_prompt: str) -> dict:
    response = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ]
    )
    return response.choices[0].message.content

user_prompt_summary = get_user_prompt_summary(
    url=plancraft.url, 
    website_contents=website_contents
)

response = call_openai(
    system_prompt=system_prompt_summary,
    user_prompt=user_prompt_summary
)

display(Markdown(response))

# Plancraft Company Overview

## Product Details
Plancraft is a comprehensive craftsman software tailored for various trades including building contractors, roofers, painters, joiners, and carpenters. The platform enhances business operations by integrating features such as:

- **Order Documents**: Quickly create offers, order confirmations, delivery notes, and invoices.
- **Craftsman Cloud**: Access data anytime and anywhere via the cloud.
- **Mobile Measurement**: Facilitates on-site measurements.
- **Financial Administration**: Manage payments remotely.
- **Time Recording**: Monitor working hours in real-time.
- **Master Data Management**: Simplified data handling.
- **Post-Calculation**: Analyze and determine profitable pricing for services.
- **Planning Board**: Streamlined project and deployment planning.

Plancraft primarily focuses on digitizing processes in craft businesses, leading to significant time savings for users. According to success stories featured on their site, users have reported saving considerable hours weekly thanks to Plancraft’s efficiency.

## Company Culture
Plancraft fosters a vibrant company culture centered around teamwork, humility, and innovation. Their mission emphasizes creating a supportive environment for craftsmen to thrive. The company champions collaboration, continuous learning, and a hands-on approach in addressing user needs. The team values direct contact and responsiveness, ensuring clients receive prompt support.

### Team Values
- **Together**: Emphasis on collaboration and mutual respect.
- **Humble**: Acknowledgment of everyday authenticity and growth.
- **Stoked**: Passionate about redefining craftsmanship through digital tools.

## Founders
Plancraft was founded in 2020 by a trio of dedicated entrepreneurs:
- **Julian** - CEO: Interaction and user digitization champion.
- **Richard** - CTO: Oversees technical implementations.
- **Alexander** - CPO: Craftsman background influencing application development.

Contact Information:
- **Sales Team**: [sales@plancraft.de](mailto:sales@plancraft.de)
- **Phone**: +49 40 32 89 02 43 0

For more insights, you can book a demo or start a free trial on their website. Plancraft is committed to redefining craftsmanship through their innovative software solutions.

## Combine functionality into one function call
Awesome!! To make this a bit simpler, let's combine these function calls into a single one.

In [164]:
def create_company_overview(url: str):
    website = Website(url)
    
    user_prompt_links = create_user_prompt_links(website = website)
    links = call_openai_json(
        system_prompt=system_prompt_links,
        user_prompt=user_prompt_links
    )
    
    websites = scrape_links(homepage=website, links=links)
    website_contents = "\n\n".join(website.get_contents() for website in websites)
    user_prompt_summary = get_user_prompt_summary(
        url=website.url, 
        website_contents=website_contents
    )
    response = call_openai(
        system_prompt=system_prompt_summary,
        user_prompt=user_prompt_summary
    )

    display(Markdown(response))

In [165]:
create_company_overview("https://neuroflash.com/")

# neuroflash Overview

## Company Product
neuroflash is a leading AI-based content creation platform in Europe, specializing in the generation of both textual and visual content. The core products offered by neuroflash include:

- **Content Generator & Editor**: An intelligent assistant for generating and editing content with integrated SEO analysis and over 100 text templates.
- **ChatFlash**: An AI chatbot that generates responses based on the user's knowledge and established writing style.
- **Image Generator**: A tool for creating copyright-free images in various styles.
- **PerformanceFlash**: Predicts and optimizes content performance, especially short form copy.
- **Brand Hub**: A centralized platform for saving brand voices and style guidelines for streamlined content creation.
- **neuroflash API**: Allows integration of neuroflash's capabilities into other applications.

The platform enables users to create, personalize, and optimize content efficiently, promoting brand consistency and originality across multiple channels.

## Company Culture
neuroflash prides itself on fostering a diverse and collaborative work culture, allowing global employees to work from anywhere. Their team reflects a commitment to balancing professionalism with creativity, encouraging flexibility such as remote work and personal growth opportunities. The company values customer-centric solutions and practices, ensuring that their offerings meet specific needs effectively. 

## Founders
neuroflash was founded in July 2021 by:

1. **Henrik Büning** - Co-CEO & Co-Founder
   - [LinkedIn Profile](https://www.linkedin.com/in/henrik-buening-876b501a1/)
   
2. **Henrik Roth** - Co-CEO & Co-Founder
   - [LinkedIn Profile](https://www.linkedin.com/in/henrik-roth-6b58b7195/)
   
3. **Jonathan Mall** - CIO & Co-Founder
   - [LinkedIn Profile](https://www.linkedin.com/in/jonathan-mall-747664a8/)

For inquiries, please connect with them through their LinkedIn profiles or through the official neuroflash website for more formal communication channels. 

## Contact Information
- **Website**: [neuroflash.com](https://neuroflash.com/)
- **Support Email**: Available through the Help Center on their website.

This summary encapsulates the essence of neuroflash as a prominent AI content generation company, emphasizing their innovative solutions, cultural values, and leadership.

## Improving the solution by wrapping it with a simple UI
How cool is that? Just showcases how easy it is to implement simple use cases, all without training and maintaining your own models. To make this a bit nicer, let's wrap a lean UI around it to create an MVP application. We will use Gradio for that.

First, let's rewrite that second OpenAI API to stream back the result to make it a bit more like ChatGPT, which is OpenAI's user interface to use these models. We will use the API response to render the result a bit more nicely.

In [166]:
def call_openai_stream(system_prompt: str, user_prompt: str) -> Generator[dict, None, None]:
    return openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        stream=True
    )

Let's now recreate the create_company_overview function to make it stream the overview. We can use it as a callback for our UI.

In [167]:
def stream_company_overview(url: str):
    website = Website(url)
    
    user_prompt_links = create_user_prompt_links(website = website)
    links = call_openai_json(
        system_prompt=system_prompt_links,
        user_prompt=user_prompt_links
    )
    
    websites = scrape_links(homepage = website, links = links)
    website_contents = "\n\n".join(website.get_contents() for website in websites)
    user_prompt_summary = get_user_prompt_summary(
        url=website.url, 
        website_contents=website_contents
    )
    stream = call_openai_stream(
        system_prompt=system_prompt_summary,
        user_prompt=user_prompt_summary
    )

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response

## Creating the UI
Finally, let's create our Gradio UI to pass comany website URLs to the function and stream back the results.

In [168]:
import gradio as gr

gr.Interface(
    fn=stream_company_overview,
    inputs=[gr.Textbox(label="Enter company website URL:", lines=1)],
    outputs=[gr.Markdown(label="Company overview:")],
    allow_flagging="never"
).launch()

Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.
